<a href="https://colab.research.google.com/github/gugui-python/movies-box-office-dataset/blob/main/ProyectoDSParteIII%2BPerelloLund.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Curso: Data Science 1

Entrega 3 - Proyecto Final

Alumna: María Agustina Perelló Lund


**Abstract**

Este proyecto realiza un análisis de la base de datos “Enhanced Box Office Data (2000–2024)”, que contiene información sobre películas estrenadas entre 2000 y 2024, e incluye datos de recaudación mundial, géneros de las películas, idioma en el que se grabó, calificaciones y más.
Tiene más de 2000 registros y 13 variables (columnas), y vamos a tratar de descubrir patrones en el éxito de las películas. Algunas variables clave y las que vamos a utilizar, son la recaudación mundial (`$Worldwide`), la calificación promedio de audiencia (`Rating`) y los géneros (`Genres`).

**Objetivo**

Mi objetivo es investigar cómo diferentes variables, como el rating y el género de las películas, influyen en su recaudación mundial. Esto no solo ayuda a entender mejor qué hace que una película sea exitosa, sino también a predecir qué factores contribuyen a ese éxito. Los insights obtenidos podrían ayudar a los estudios de cine a tomar decisiones informadas sobre la inversión en producciones, la elección de géneros y la asignación de recursos para marketing.

**Contexto comercial**

El análisis que estoy realizando es muy útil para la industria del cine porque permite entender cómo las decisiones de inversión, marketing, y selección de géneros pueden afectar el éxito de una película. Los estudios de cine pueden usar estos insights para elegir los géneros más rentables, ajustar sus estrategias de marketing o incluso decidir en qué tipo de producciones invertir para maximizar su retorno.


**Contexto analítico**

Mi análisis se centra en explorar cómo variables como el rating y la recaudación mundial se relacionan, además de cómo los géneros impactan en esos dos factores. Las visualizaciones (como los boxplots, scatterplots, etc.) en la primera entrega me ayudaron a identificar patrones clave, como la relación positiva entre el rating y la recaudación, lo que es muy útil para los estudios que buscan maximizar los ingresos de sus películas. Este análisis conecta directamente con el objetivo de predecir qué películas podrían ser más exitosas según su género y rating.

In [4]:
# CARGA Y VISTA PREVIA DEL DATASET
# Importación de Librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import SelectKBest, f_regression

# Cargar dataset
url = "https://raw.githubusercontent.com/gugui-python/movies-box-office-dataset/main/enhanced_box_office_data(2000-2024)u.csv"
df_movies = pd.read_csv(url)

# Vista previa
df_movies.head()
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Rank                  5000 non-null   int64  
 1   Release Group         5000 non-null   object 
 2   $Worldwide            5000 non-null   float64
 3   $Domestic             5000 non-null   float64
 4   Domestic %            5000 non-null   float64
 5   $Foreign              5000 non-null   float64
 6   Foreign %             5000 non-null   float64
 7   Year                  5000 non-null   int64  
 8   Genres                4822 non-null   object 
 9   Rating                4830 non-null   object 
 10  Vote_Count            4830 non-null   float64
 11  Original_Language     4830 non-null   object 
 12  Production_Countries  4800 non-null   object 
dtypes: float64(6), int64(2), object(5)
memory usage: 507.9+ KB


In [5]:
# Análisis de valores perdidos por columna
df_movies.isnull().sum()

,0
Rank,0
Release Group,0
$Worldwide,0
$Domestic,0
Domestic %,0
$Foreign,0
Foreign %,0
Year,0
Genres,178
Rating,170


De las columnas que nos interesan, las que tienen valores nulos son los géneros de las películas y el rating. Considero que para los gráficos, es preferible borrar estas columnas directamente.

- **Rating:** Esta variable es fundamental para muchas de las visualizaciones y relaciones planteadas. Sin estos datos, no podríamos analizar correctamente las calificaciones promedio.
- **Genres:** Dado que parte del análisis compara películas por género, también es importante contar con este dato completo. Aunque se podría imputar, en este caso decidí eliminar esas filas ya que la cantidad de nulos era pequeña en relación al total.

Además, vamos a transformar la columna `Rating`, que estaba en formato texto como "6.3/10", para convertirla en un número decimal (por ejemplo, 6.3), para poder realizar cálculos y visualizaciones numéricas.


In [15]:
# Eliminar filas con valores nulos en Rating y Genres y crear copia
df_movies = df_movies.dropna(subset=['Rating', 'Genres']).copy()

# Transformar Rating a número decimal
df_movies['Rating'] = df_movies['Rating'].astype(str).str.split('/').str[0]
df_movies['Rating'] = pd.to_numeric(df_movies['Rating'], errors='coerce')  # fuerza conversión numérica segura

# Verificar que estén bien los valores
df_movies['Rating'].head()


,Rating
0,6.126
1,8.217
2,7.663
3,6.450
4,6.544


In [16]:
# Vista rápida del dataset después de la limpieza
df_movies.head()

,Rank,Release Group,$Worldwide,$Domestic,Domestic %,$Foreign,Foreign %,Year,Genres,Rating,Vote_Count,Original_Language,Production_Countries
0,1,Mission: Impossible II,546388108.0,215409889.0,39.4,330978219.0,60.6,2000,"Adventure, Action, Thriller",6.126,6741.0,en,United States of America
1,2,Gladiator,460583960.0,187705427.0,40.8,272878533.0,59.2,2000,"Action, Drama, Adventure",8.217,19032.0,en,"United Kingdom, United States of America"
2,3,Cast Away,429632142.0,233632142.0,54.4,196000000.0,45.6,2000,"Adventure, Drama",7.663,11403.0,en,United States of America
3,4,What Women Want,374111707.0,182811707.0,48.9,191300000.0,51.1,2000,"Comedy, Romance",6.450,3944.0,en,"United Kingdom, United States of America"
4,5,Dinosaur,349822765.0,137748063.0,39.4,212074702.0,60.6,2000,"Animation, Family, Adventure",6.544,2530.0,en,United States of America


En la entrega anterior, nos dimoscuenta que una película puede pertenecer a varios géneros. Para mejorar la precisión del análisis, ahora vamos a separar los géneros en filas únicas, permitiendo analizar cómo cada género individual impacta en la recaudación y el rating.


In [17]:
df_movies['Genres'] = df_movies['Genres'].fillna('Unknown')
df_genres = df_movies.assign(Genre=df_movies['Genres'].str.split(',')).explode('Genre').reset_index(drop=True)

# Vista del nuevo dataset
df_genres.head()


,Rank,Release Group,$Worldwide,$Domestic,Domestic %,$Foreign,Foreign %,Year,Genres,Rating,Vote_Count,Original_Language,Production_Countries,Genre
0,1,Mission: Impossible II,546388108.0,215409889.0,39.4,330978219.0,60.6,2000,"Adventure, Action, Thriller",6.126,6741.0,en,United States of America,Adventure
1,1,Mission: Impossible II,546388108.0,215409889.0,39.4,330978219.0,60.6,2000,"Adventure, Action, Thriller",6.126,6741.0,en,United States of America,Action
2,1,Mission: Impossible II,546388108.0,215409889.0,39.4,330978219.0,60.6,2000,"Adventure, Action, Thriller",6.126,6741.0,en,United States of America,Thriller
3,2,Gladiator,460583960.0,187705427.0,40.8,272878533.0,59.2,2000,"Action, Drama, Adventure",8.217,19032.0,en,"United Kingdom, United States of America",Action
4,2,Gladiator,460583960.0,187705427.0,40.8,272878533.0,59.2,2000,"Action, Drama, Adventure",8.217,19032.0,en,"United Kingdom, United States of America",Drama


In [20]:
#Feature Selection -> usamos Random Forest

# Definir variables independientes (X) y dependiente (y)
X = df_movies[['Rating', 'Year']]  # Usamos 'Rating' y 'Year'
y = df_movies['$Worldwide']  # Variable dependiente

# Selección de las mejores características
selector = SelectKBest(score_func=f_regression, k='all')
X_selected = selector.fit_transform(X, y)

# Mostrar las puntuaciones de las características seleccionadas
scores = pd.DataFrame({'Feature': X.columns, 'Score': selector.scores_})
print(scores)


  Feature       Score
0  Rating  168.976649
1    Year   18.936139


Podemos ver que el Rating tiene una puntuación mucho más alta que Year (año de estreno), lo que indica que Rating tiene una mayor relación con la variable dependiente $Worldwide (recaudación mundial). Esto sugiere que el rating tiene una mayor influencia en la recaudación mundial en comparación con el año de estreno de la película.

In [21]:
#Regresión (entrenamiento del modelo) -> usamos Random Forest Regressor

# Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

# Evaluación del modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

# Mostrar métricas de evaluación
print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")
print(f"Mean Absolute Error: {mae}")


Mean Squared Error: 4.701802383784733e+16
R2 Score: -0.2863163770473909
Mean Absolute Error: 119267739.93746288


Los resultados muestran que el modelo de regresión no está teniendo un buen desempeño.

Mean Squared Error (MSE): Un valor tan grande indica que el modelo está cometiendo errores muy grandes. Idealmente, queremos que esta cifra sea lo más chica posible.

R² Score: Un R² negativo significa que el modelo no está explicando bien la variabilidad de los datos, y está haciendo un peor trabajo que un modelo que simplemente predice la media de la variable dependiente. Esto sugiere que el modelo está lejos de ser efectivo.

Mean Absolute Error (MAE): El valor de MAE es también bastante alto, lo que implica que las predicciones del modelo se desvían significativamente de los valores reales.

Vamos a probar agregando otras columnas adicionales a X (las variables independientes), para ver si influyen en la recaudación mundial. Además, voy a intentar aplicar una transformación logarítmica para mejorar la predicción y evitar el sesgo de los valores más altos.

In [22]:
# Incluir más características (aumentar X)
X = df_movies[['Rating', 'Year', 'Domestic %', 'Foreign %', 'Vote_Count']]  # Agregamos características

# Transformar la variable dependiente (Worldwide) usando logaritmo
y = np.log1p(df_movies['$Worldwide'])  # Aplicamos log1p para manejar valores 0 y grandes

# Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Crear y entrenar el modelo
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

# Evaluación
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R2 Score: {r2}")
print(f"Mean Absolute Error: {mae}")


Mean Squared Error: 0.5025331549929317
R2 Score: 0.6368463184230391
Mean Absolute Error: 0.5397079067395437


Mean Squared Error (MSE): 0.50. El error cuadrático medio es bastante bajo, lo que significa que las predicciones no están tan lejos de los valores reales.

R² Score: 0.64. Esto indica que alrededor del 64% de la varianza en $Worldwide es explicada por las características que hemos seleccionado.

Mean Absolute Error (MAE): 0.54. Esto significa que, en promedio, la predicción está a un 54% del valor real.

**CONCLUSIÓN:**

Este proyecto intentó entender los factores que influyen en la recaudación mundial de las películas, utilizando un conjunto de datos de 5000 películas, con variables como Rating, Año de Estreno, Recaudación en USD y más. Después de realizar un análisis exploratorio, limpieza de datos y transformación de las variables, implementamos un modelo de Random Forest Regressor para predecir la recaudación mundial.

Los resultados finales del modelo (luego de iterar, ya que al principio fueron distintos) son los siguientes:

- Mean Squared Error (MSE): 0.5025

- R2 Score: 0.6368

- Mean Absolute Error (MAE): 0.5397

Estos resultados indican que el modelo tiene una capacidad moderada para predecir la recaudación mundial de las películas. Un R2 Score de 0.64 sugiere que el modelo explica aproximadamente el 64% de la variabilidad de la recaudación mundial, que considero un buen comienzo.

En el análisis, se incluyeron características adicionales a las pensadas inicialmente:

Porcentaje de recaudación nacional y extranjera: Estas variables proporcionaron un contexto adicional sobre el origen de la recaudación, lo que permitió al modelo capturar diferencias importantes entre películas que tuvieron un mayor éxito en el mercado nacional versus el internacional.

Conteo de votos: Esta característica refleja el interés y la popularidad general de una película, lo que tiene un impacto directo en la recaudación mundial.

La adición de estas características resultó en una mejora significativa en la capacidad predictiva del modelo, especialmente al considerar que la recaudación mundial depende de factores múltiples y complejos. Los modelos que solo usaban rating y año de estreno generaron resultados insatisfactorios, con un R2 Score bajo, lo que confirmaba la necesidad de incorporar más variables para una predicción más precisa.

En conclusión, agregar más variables nos permitió una mejora considerable en la predicción de la recaudación mundial, mejorando los resultados obtenidos solo con rating y año de estreno. Aunque el modelo actual muestra un desempeño razonable, aún hay espacio para optimizar el rendimiento y explorar más características que podrían mejorar las predicciones. El siguiente paso debería centrarse en la optimización del modelo y la expansión de las variables para una predicción aún más precisa.